In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import sympy as sp
from scipy.signal import TransferFunction, bode, lsim
from scipy import signal
import control as control
from IPython.display import Math, display, Image, Markdown, Latex

# Vergleich Einstellverfahren

In [ ]:
imag = Image("../../images/zweitank.png", width=800)
display(imag)

## 2-Tanksystem

- lineares Modell im Zeitbereich
    \begin{align*}
        \dot{\tilde{z}}_1(t) & = a_{11} \tilde{z}_1(t) + b \tilde{u}_{\text{A}}(t)\\
        \dot{\tilde{z}}_2(t) & = a_{21} \tilde{z}_1(t) + a_{22} \tilde{z}_2(t) 
    \end{align*}
    mit Ausgang $y(t) = z_2(t)$ bzw.
    \begin{align*}
        T_1T_2\ddot{\tilde{z}}_2(t) + (T_1 + T_2)\dot{\tilde{z}}_2(t) + \tilde{z}_2(t) & = K \tilde{u}_{\text{A}}(t)
    \end{align*}
- lineares Modell im Bildbereich
    \begin{align*}
        P(s) &= \frac{K_\mathrm{S}}{(1 + T_1 s) (1 + T_2 s)} = \frac{0.15}{(1 + 68.21 s) (1 + 34.57 s)}
    \end{align*}

In [ ]:
f = np.logspace(-5, 1)
time = np.linspace(0, 750, 751)

In [ ]:
T1 = 68.21
T2 = 34.57
KS = 0.15
sys = control.TransferFunction([KS], [T1 * T2, T1 + T2, 1])
wSys, magSys, phaseSys = bode(sys.returnScipySignalLTI()[0][0], f)

- Sprungangwort

In [ ]:
u = np.ones(len(time)) * 6
tSimSys, ySimSys, _ = signal.lsim(sys.returnScipySignalLTI()[0][0], U=u, T=time)

## Regler

* da System 2.Ordnung $\Rightarrow$ PI-Regler
\begin{align*}
    R(s) & = K_{\mathrm{R}} \frac{1 + T_{\mathrm{N}} s}{T_{\mathrm{N}} s}
\end{align*}

* offener Kreis
\begin{align*}
    G_0(s) & = R(s) P(s) = K_{\mathrm{R}} \frac{1 + T_{\mathrm{N}} s}{T_{\mathrm{N}} s}\frac{K_\mathrm{S}}{(1 + T_1 s) (1 + T_2 s)}
\end{align*}


### Verfahren nach Reinisch

In [ ]:
a = 2
Tn = T1
TSum = T2
KP = Tn / (a * KS * TSum)
display(Markdown(rf"""
$K_\mathrm{{P}} = {KP},\quad T_\mathrm{{N}} = {Tn},\quad T_\mathrm{{\sum}} = {TSum}$
"""))

In [ ]:
ctrlReinisch = control.TransferFunction([KP * Tn, KP],[Tn, 0])
wCtrlReinisch, magCtrlReinisch, phaseCtrlReinisch = bode(ctrlReinisch.returnScipySignalLTI()[0][0], f)
display(Latex("$R(s) = $"))
ctrlReinisch

In [ ]:
G0Reinisch = (ctrlReinisch * sys).minreal()
wG0Reinisch, magG0Reinisch, phaseG0Reinisch = bode(G0Reinisch.returnScipySignalLTI()[0][0], f)
display(Latex("$G_0(s) = R(s) P(s) = $"))
G0Reinisch

In [ ]:
GReinisch = (G0Reinisch / (1 + G0Reinisch)).minreal()
wGReinisch, magGReinisch, phaseGReinisch = bode(GReinisch.returnScipySignalLTI()[0][0], f)
display(Latex("$G(s) = \\frac{G_0(s)}{1 + G_0(s)} = \\frac{R(s) P(s)}{1 + R(s) P(s)} = $"))
GReinisch

In [ ]:
plt.close()

fig1, (axes10, axes20) = plt.subplots(2, 1, figsize=(12,7))

axes10.semilogx(wSys, magSys, label='Prozess')
axes10.semilogx(wCtrlReinisch, magCtrlReinisch, label='Regler')
axes10.semilogx(wG0Reinisch, magG0Reinisch, label='offener Kreis')
axes10.semilogx(wGReinisch, magGReinisch, label='geschlossener Kreis')
axes20.semilogx(wSys, phaseSys)
axes20.semilogx(wCtrlReinisch, phaseCtrlReinisch)
axes20.semilogx(wG0Reinisch, phaseG0Reinisch)
axes20.semilogx(wGReinisch, phaseGReinisch)

axes20.set_xlabel(r"$\omega$")
axes10.set_ylabel(r"$|G(j \omega)|_{\mathrm{dB}}$")
axes20.set_ylabel(r"$\mathrm{arg}\, G(j\omega)/^{\circ}$")

axes10.grid() 
axes20.grid()

handlesAx, labelsAx = axes10.get_legend_handles_labels()
fig1.legend([handle for i, handle in enumerate(handlesAx)],
            [label for i, label in enumerate(labelsAx)],
            bbox_to_anchor=(0.125, 0.90, 0.775, .15), loc=3,
            ncol=4, mode="expand", borderaxespad=0., framealpha=0.5)
plt.show()

### Symmetrisches Optimum

In [ ]:
a = 2
TSum = T2
Tn = a ** 2 * TSum
KP = Tn / (a * KS * TSum)
display(Markdown(rf"""
$K_\mathrm{{P}} = {KP},\quad T_\mathrm{{N}} = {Tn},\quad T_\mathrm{{\sum}} = {TSum}$
"""))

In [ ]:
ctrlSym = control.TransferFunction([KP * Tn, KP],[Tn, 0])
wCtrlSym, magCtrlSym, phaseCtrlSym = bode(ctrlSym.returnScipySignalLTI()[0][0], f)
display(Latex("$R(s) = $"))
ctrlSym

In [ ]:
G0Sym = (ctrlSym * sys).minreal()
wG0Sym, magG0Sym, phaseG0Sym = bode(G0Sym.returnScipySignalLTI()[0][0], f)
display(Latex("$G_0(s) = R(s) P(s) = $"))
G0Sym

In [ ]:
GSym = (G0Sym / (1 + G0Sym)).minreal()
wGSym, magGSym, phaseGSym = bode(GSym.returnScipySignalLTI()[0][0], f)
display(Latex("$G(s) = \\frac{G_0(s)}{1 + G_0(s)} = \\frac{R(s) P(s)}{1 + R(s) P(s)} = $"))
GSym

In [ ]:
plt.close()

fig1, (axes10, axes20) = plt.subplots(2, 1, figsize=(12,7))

axes10.semilogx(wSys, magSys, label='Prozess')
axes10.semilogx(wCtrlSym, magCtrlSym, label='Regler')
axes10.semilogx(wG0Sym, magG0Sym, label='offener Kreis')
axes10.semilogx(wGSym, magGSym, label='geschlossener Kreis')
axes20.semilogx(wSys, phaseSys)
axes20.semilogx(wCtrlSym, phaseCtrlSym)
axes20.semilogx(wG0Sym, phaseG0Sym)
axes20.semilogx(wGSym, phaseGSym)

axes20.set_xlabel(r"$\omega$")
axes10.set_ylabel(r"$|G(j \omega)|_{\mathrm{dB}}$")
axes20.set_ylabel(r"$\mathrm{arg}\, G(j\omega)/^{\circ}$")

axes10.grid() 
axes20.grid()

handlesAx, labelsAx = axes10.get_legend_handles_labels()
fig1.legend([handle for i, handle in enumerate(handlesAx)],
            [label for i, label in enumerate(labelsAx)],
            bbox_to_anchor=(0.125, 0.90, 0.775, .15), loc=3,
            ncol=4, mode="expand", borderaxespad=0., framealpha=0.5)
plt.show()

## Vergleich

In [ ]:
tSimCtrlReinisch, ySimCtrlReinisch, _ = signal.lsim(GReinisch.returnScipySignalLTI()[0][0], U=np.ones(len(time)) * 0.9, T=time)
tSimCtrlSym, ySimCtrlSym, _ = signal.lsim(GSym.returnScipySignalLTI()[0][0], U=np.ones(len(time)) * 0.9, T=time)

In [ ]:
plt.close()

fig1, axes10 = plt.subplots(1, 1, figsize=(12,7))

axes10.plot(time, np.ones(len(time)) * 0.9, label='Sollwert')
axes10.plot(tSimSys, ySimSys, label='Prozess')
axes10.plot(tSimCtrlReinisch, ySimCtrlReinisch, label='Reinisch')
axes10.plot(tSimCtrlSym, ySimCtrlSym, label='Symmetrisches Optimum')

axes11 = axes10.twinx() 

axes11.plot(time, u, color='C4')
axes11.tick_params(axis='y', labelcolor='C4')

axes10.set_ylabel(r'$\tilde{z}_2$ in m')
axes11.set_ylabel(r'$\tilde{u}_{\mathrm{A}}$ in V', color='C4')
axes10.set_xlabel(r'$t$ in s')

handlesAx1, labelsAx1 = axes10.get_legend_handles_labels()
axes10.legend([handle for i, handle in enumerate(handlesAx1)],
              [label for i, label in enumerate(labelsAx1)],
              bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
              ncol=9, mode="expand", borderaxespad=0., framealpha=0.5)

axes10.grid()
plt.show()